### Versión 1

In [12]:
import pandas as pd
from scipy.spatial.distance import euclidean

# === PARÁMETROS ===
dias_totales = 30
aclimatacion_min_dias = 3

# Coordenadas de polígonos (18 = almacén)
poligonos_coords = {
    1: (0, 0), 2: (2, 1), 3: (4, 3), 4: (5, 5), 5: (3, 6),
    6: (1, 7), 7: (0, 5), 8: (2, 4), 9: (6, 2), 10: (8, 3),
    11: (10, 2), 12: (9, 5), 13: (7, 6), 14: (5, 8), 15: (3, 9),
    16: (1, 9), 17: (0, 8), 18: (2, 7)
}
almacen_coord = poligonos_coords[18]

# Demanda por especie y polígono
demanda_poligonos = {
    "Encino": {
        1: 300, 2: 200, 3: 250, 4: 400, 5: 300, 6: 350,
        7: 150, 8: 300, 9: 200, 10: 250, 11: 300, 12: 250,
        13: 250, 14: 300, 15: 300, 16: 200, 17: 100
    },
    "Pino": {
        1: 200, 2: 150, 3: 200, 4: 300, 5: 250, 6: 300,
        7: 100, 8: 200, 9: 150, 10: 200, 11: 200, 12: 200,
        13: 200, 14: 250, 15: 200, 16: 150, 17: 100
    }
}

# Proveedores
proveedores = {
    "Prov1": {"Encino": {"costo": 8, "max_oferta": 3000}, "Pino": {"costo": 6, "max_oferta": 2500}},
    "Prov2": {"Encino": {"costo": 7, "max_oferta": 2500}, "Pino": {"costo": 5, "max_oferta": 2500}}
}

# Logística
capacidad_total_camion = 800
costo_transporte = 4500
velocidad = 60 # km/h
carga_min = 30
descarga_min = 30
jornada_min = 360
espacio_max_almacen = 8000
costo_plantacion = 20

# === ESTADOS ===
inventario = {esp: [0] * (dias_totales + 1) for esp in demanda_poligonos}
disponibles = {esp: [0] * (dias_totales + 1) for esp in demanda_poligonos}
oferta_usada = {p: {esp: 0 for esp in demanda_poligonos} for p in proveedores}
demanda_restante = {esp: demanda_poligonos[esp].copy() for esp in demanda_poligonos}
compras = []
entregas = []
rutas_totales = []

def tiempo_entre(p1, p2):
    dist = euclidean(poligonos_coords[p1], poligonos_coords[p2])
    return (dist / velocidad) * 60

for dia in range(dias_totales):
    for esp in demanda_poligonos:
        if dia > 0:
            inventario[esp][dia] += inventario[esp][dia - 1]
            disponibles[esp][dia] += disponibles[esp][dia - 1]
        if dia >= aclimatacion_min_dias:
            disponibles[esp][dia] += inventario[esp][dia - aclimatacion_min_dias]

    entregado_hoy = {esp: 0 for esp in demanda_poligonos}
    tiempo_total_dia = 0
    rutas_dia = []

    while tiempo_total_dia < jornada_min:
        ruta = [18]
        tiempo_ruta = carga_min
        carga_actual = 0
        entrega_ruta = []
        candidatos = sorted(
            {pid for esp in demanda_restante for pid, d in demanda_restante[esp].items() if d > 0},
            key=lambda pid: -euclidean(almacen_coord, poligonos_coords[pid])
        )
        print(candidatos)

        while candidatos:
            last_node = ruta[-1]
            next_node = None
            for pid in candidatos:
                t_extra = tiempo_entre(last_node, pid) + descarga_min + tiempo_entre(pid, 18)
                if tiempo_total_dia + tiempo_ruta + t_extra > jornada_min:
                    continue

                entrega_especie = {}
                total_entrega = 0
                for esp in demanda_poligonos:
                    disp = disponibles[esp][dia]
                    dem = demanda_restante[esp].get(pid, 0)
                    qty = min(disp, dem, capacidad_total_camion - carga_actual - total_entrega)
                    if qty > 0:
                        entrega_especie[esp] = qty
                        total_entrega += qty

                if total_entrega > 0:
                    next_node = pid
                    break

            if not next_node:
                break

            ruta.append(next_node)
            tiempo_ruta += tiempo_entre(ruta[-2], next_node) + descarga_min
            entrega_ruta.append((next_node, entrega_especie))
            for esp, qty in entrega_especie.items():
                disponibles[esp][dia] -= qty
                demanda_restante[esp][next_node] -= qty
                entregado_hoy[esp] += qty
            carga_actual += sum(entrega_especie.values())
            candidatos.remove(next_node)

        if len(ruta) == 1:
            break

        tiempo_ruta += tiempo_entre(ruta[-1], 18)
        tiempo_total_dia += tiempo_ruta

        rutas_dia.append({
            "Día": dia,
            "Ruta": " → ".join(map(str, ruta + [18])),
            "Duración total (min)": round(tiempo_ruta),
            "Unidades entregadas": carga_actual,
            "Detalle por nodo": entrega_ruta
        })

    for esp in demanda_poligonos:
        inventario[esp][dia] -= entregado_hoy[esp]
        if entregado_hoy[esp] > 0:
            entregas.append({
                "Especie": esp,
                "Día entrega": dia,
                "Cantidad entregada": entregado_hoy[esp],
                "Costo plantación": entregado_hoy[esp] * costo_plantacion
            })

    for esp in demanda_poligonos:
        if dia + aclimatacion_min_dias >= dias_totales:
            continue
        total_demandado = sum(demanda_poligonos[esp].values())
        total_comprado = sum([c["Cantidad"] for c in compras if c["Especie"] == esp])
        restante_global = total_demandado - total_comprado
        espacio_disp = espacio_max_almacen - inventario[esp][dia]
        max_posible = min(restante_global, espacio_disp)

        opciones = [(p, d[esp]["costo"], d[esp]["max_oferta"] - oferta_usada[p][esp])
                    for p, d in proveedores.items() if esp in d and d[esp]["max_oferta"] - oferta_usada[p][esp] > 0]
        if opciones and max_posible > 0:
            opciones.sort(key=lambda x: x[1])
            prov, costo, oferta_rest = opciones[0]
            cantidad = min(oferta_rest, max_posible)

            inventario[esp][dia + 1] += cantidad
            oferta_usada[prov][esp] += cantidad
            compras.append({
                "Especie": esp,
                "Día pedido": dia,
                "Proveedor": prov,
                "Cantidad": cantidad,
                "Costo compra": cantidad * costo,
                "Costo transporte": costo_transporte
            })

    rutas_totales += rutas_dia

# === EXPORTAR ===
df_compras = pd.DataFrame(compras)
df_entregas = pd.DataFrame(entregas)
df_rutas = pd.DataFrame(rutas_totales)

print("=== COMPRAS ===")
print(df_compras)
print("\n=== ENTREGAS ===")
print(df_entregas)
print("\n=== RUTAS ===")
print(df_rutas)


[11, 1, 12, 10, 9, 2, 13, 3, 4, 14, 8, 7, 15, 16, 17, 5, 6]
[11, 1, 12, 10, 9, 2, 13, 3, 4, 14, 8, 7, 15, 16, 17, 5, 6]
[11, 1, 12, 10, 9, 2, 13, 3, 4, 14, 8, 7, 15, 16, 17, 5, 6]
[11, 1, 12, 10, 9, 2, 13, 3, 4, 14, 8, 7, 15, 16, 17, 5, 6]
[11, 1, 12, 10, 9, 2, 13, 3, 4, 14, 8, 7, 15, 16, 17, 5, 6]
[1, 12, 10, 9, 2, 13, 3, 4, 14, 8, 7, 15, 16, 17, 5, 6]
[10, 9, 2, 13, 3, 4, 14, 8, 7, 15, 16, 17, 5, 6]
[9, 2, 13, 3, 4, 14, 8, 7, 15, 16, 17, 5, 6]
[9, 2, 13, 3, 4, 14, 8, 7, 15, 16, 17, 5, 6]
[13, 3, 4, 14, 8, 7, 15, 16, 17, 5, 6]
[4, 14, 8, 7, 15, 16, 17, 5, 6]
[14, 8, 7, 15, 16, 17, 5, 6]
[14, 8, 7, 15, 16, 17, 5, 6]
[8, 7, 15, 16, 17, 5, 6]
[15, 16, 17, 5, 6]
[17, 5, 6]
[17, 5, 6]
[6]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
=== COMPRAS ===
  Especie  Día pedido Proveedor  Cantidad  Costo compra  Costo transporte
0  Encino           0     Prov2      2500         17500              4500
1    Pino           0     Prov2      2500         12500              4500

In [ ]:
df_rutas

,Día,Ruta,Duración total (min),Unidades entregadas,Detalle por nodo
0,4,18 → 11 → 1 → 18,117,800,"[(11, {'Encino': 300, 'Pino': 200}), (1, {'Enc..."
1,4,18 → 1 → 12 → 10 → 18,147,800,"[(1, {'Pino': 200}), (12, {'Encino': 250, 'Pin..."
2,4,18 → 10 → 18,74,300,"[(10, {'Encino': 100, 'Pino': 200})]"
3,5,18 → 9 → 2 → 13 → 18,143,800,"[(9, {'Encino': 200, 'Pino': 150}), (2, {'Enci..."
4,5,18 → 13 → 3 → 18,104,800,"[(13, {'Encino': 150, 'Pino': 200}), (3, {'Enc..."
5,5,18 → 4 → 14 → 18,100,800,"[(4, {'Encino': 400, 'Pino': 300}), (14, {'Enc..."
6,6,18 → 14 → 8 → 18,101,800,"[(14, {'Encino': 200, 'Pino': 250}), (8, {'Enc..."
7,6,18 → 8 → 7 → 15 → 18,132,800,"[(8, {'Pino': 150}), (7, {'Encino': 150, 'Pino..."
8,6,18 → 15 → 16 → 18,96,450,"[(15, {'Pino': 100}), (16, {'Encino': 200, 'Pi..."
9,7,18 → 17 → 5 → 6 → 18,129,800,"[(17, {'Encino': 100, 'Pino': 100}), (5, {'Enc..."


### Versión 2

In [1]:
import pandas as pd
from scipy.spatial.distance import euclidean

# === PARÁMETROS ===
dias_totales = 30
aclimatacion_min_dias = 3

# Coordenadas de polígonos (18 = almacén)
poligonos_coords = {
    1: (0, 0), 2: (2, 1), 3: (4, 3), 4: (5, 5), 5: (3, 6),
    6: (1, 7), 7: (0, 5), 8: (2, 4), 9: (6, 2), 10: (8, 3),
    11: (10, 2), 12: (9, 5), 13: (7, 6), 14: (5, 8), 15: (3, 9),
    16: (1, 9), 17: (0, 8), 18: (2, 7)
}
almacen_coord = poligonos_coords[18]

# Demanda por especie y polígono
demanda_poligonos = {
    "Encino": {
        1: 300, 2: 200, 3: 250, 4: 400, 5: 300, 6: 350,
        7: 150, 8: 300, 9: 200, 10: 250, 11: 300, 12: 250,
        13: 250, 14: 300, 15: 300, 16: 200, 17: 100
    },
    "Pino": {
        1: 200, 2: 150, 3: 200, 4: 300, 5: 250, 6: 300,
        7: 100, 8: 200, 9: 150, 10: 200, 11: 200, 12: 200,
        13: 200, 14: 250, 15: 200, 16: 150, 17: 100
    }
}

# Proveedores
demandas_oferta = {
    "Prov1": {"Encino": {"costo": 8, "max_oferta": 3000}, "Pino": {"costo": 6, "max_oferta": 2500}},
    "Prov2": {"Encino": {"costo": 7, "max_oferta": 2500}, "Pino": {"costo": 5, "max_oferta": 2500}}
}

# Logística
capacidad_camion = 800
costo_transporte = 4500
velocidad = 60            # km/h
tiempo_carga = 30         # minutos
tiempo_descarga = 30      # minutos
jornada_min = 360         # minutos
espacio_max_almacen = 8000
costo_plantacion = 20


def tiempo_entre(p1, p2):
    distancia = euclidean(poligonos_coords[p1], poligonos_coords[p2])
    return (distancia / velocidad) * 60


def actualizar_inventario(inventario, dia):
    if dia == 0:
        return
    for esp in inventario:
        # sumar inventario del día anterior en lugar de sobrescribirlo
        inventario[esp][dia] += inventario[esp][dia - 1]


def calcular_disponibles(inventario):
    disponibles = {esp: [0] * (dias_totales + 1) for esp in demanda_poligonos}
    for dia in range(dias_totales + 1):
        for esp in demanda_poligonos:
            if dia >= aclimatacion_min_dias:
                disponibles[esp][dia] = inventario[esp][dia - aclimatacion_min_dias]
    return disponibles


def planificar_rutas(dia, disponibles, demanda_restante):
    rutas_dia = []
    entregado = {esp: 0 for esp in demanda_poligonos}
    tiempo_total = 0

    while tiempo_total < jornada_min:
        ruta = [18]
        tiempo_ruta = tiempo_carga
        carga = 0
        detalle = []

        while True:
            last = ruta[-1]
            candidatos = [pid for esp in demanda_restante for pid, d in demanda_restante[esp].items() if d > 0]
            candidatos.sort(key=lambda pid: euclidean(poligonos_coords[last], poligonos_coords[pid]))
            encontrado = False

            for pid in candidatos:
                t_viaje = tiempo_entre(last, pid)
                t_vuelta = tiempo_entre(pid, 18)
                t_extra = t_viaje + tiempo_descarga + t_vuelta
                if tiempo_total + tiempo_ruta + t_extra > jornada_min:
                    continue

                entrega_nodo = {}
                total_nodo = 0
                for esp in demanda_poligonos:
                    disp = disponibles[esp][dia]
                    dem = demanda_restante[esp].get(pid, 0)
                    cap_rest = capacidad_camion - carga - total_nodo
                    q = min(disp, dem, cap_rest)
                    if q > 0:
                        entrega_nodo[esp] = q
                        total_nodo += q

                if total_nodo > 0:
                    ruta.append(pid)
                    tiempo_ruta += t_viaje + tiempo_descarga
                    for esp, q in entrega_nodo.items():
                        disponibles[esp][dia] -= q
                        demanda_restante[esp][pid] -= q
                        entregado[esp] += q
                    carga += total_nodo
                    detalle.append((pid, entrega_nodo))
                    encontrado = True
                    break

            if not encontrado:
                break

        if len(ruta) > 1:
            tiempo_ruta += tiempo_entre(ruta[-1], 18)
            tiempo_total += tiempo_ruta
            rutas_dia.append({
                "Día": dia,
                "Ruta": " → ".join(map(str, ruta + [18])),
                "Duración_min": round(tiempo_ruta),
                "Unidades": carga,
                "Detalle": detalle
            })
        else:
            break

    return rutas_dia, entregado


def procesar_entregas(inventario, entregas, entregado, dia):
    for esp, cantidad in entregado.items():
        if cantidad > 0:
            inventario[esp][dia] -= cantidad
            entregas.append({
                "Especie": esp,
                "Día entrega": dia,
                "Cantidad": cantidad,
                "Costo plantación": cantidad * costo_plantacion
            })


def realizar_compras(inventario, compras, oferta_usada, dia):
    proveedores_hoja = set()
    for esp in demanda_poligonos:
        if dia + aclimatacion_min_dias >= dias_totales:
            continue
        total_dem = sum(demanda_poligonos[esp].values())
        total_cmp = sum(c["Cantidad"] for c in compras if c["Especie"] == esp)
        restante = total_dem - total_cmp
        espacio_disp = espacio_max_almacen - inventario[esp][dia]
        max_posible = min(restante, espacio_disp)
        if max_posible <= 0:
            continue

        opciones = []
        for p, datos in demandas_oferta.items():
            info = datos.get(esp)
            if info and (info["max_oferta"] - oferta_usada[p][esp]) > 0:
                opciones.append((p, info["costo"], info["max_oferta"] - oferta_usada[p][esp]))
        if not opciones:
            continue
        opciones.sort(key=lambda x: x[1])
        p_sel, costo, dispo = opciones[0]
        qty = min(dispo, max_posible)
        costo_trans = costo_transporte if p_sel not in proveedores_hoja else 0

        compras.append({
            "Especie": esp,
            "Día pedido": dia,
            "Proveedor": p_sel,
            "Cantidad": qty,
            "Costo compra": qty * costo,
            "Costo transporte": costo_trans
        })
        proveedores_hoja.add(p_sel)
        oferta_usada[p_sel][esp] += qty
        inventario[esp][dia + 1] += qty


def simular():
    inventario = {esp: [0] * (dias_totales + 1) for esp in demanda_poligonos}
    demanda_restante = {esp: demanda_poligonos[esp].copy() for esp in demanda_poligonos}
    ofertas_usadas = {p: {esp: 0 for esp in demanda_poligonos} for p in demandas_oferta}
    compras = []
    entregas = []
    rutas = []

    for dia in range(dias_totales):
        actualizar_inventario(inventario, dia)
        disponibles = calcular_disponibles(inventario)
        rutas_dia, entregado = planificar_rutas(dia, disponibles, demanda_restante)
        rutas.extend(rutas_dia)
        procesar_entregas(inventario, entregas, entregado, dia)
        realizar_compras(inventario, compras, ofertas_usadas, dia)

    return pd.DataFrame(compras), pd.DataFrame(entregas), pd.DataFrame(rutas)

if __name__ == '__main__':
    df_compras, df_entregas, df_rutas = simular()
    print("=== COMPRAS ===\n", df_compras)
    print("\n=== ENTREGAS ===\n", df_entregas)
    print("\n=== RUTAS ===\n", df_rutas)


=== COMPRAS ===
   Especie  Día pedido Proveedor  Cantidad  Costo compra  Costo transporte
0  Encino           0     Prov2      2500         17500              4500
1    Pino           0     Prov2      2500         12500                 0
2  Encino           1     Prov1      1900         15200              4500
3    Pino           1     Prov1       850          5100                 0

=== ENTREGAS ===
   Especie  Día entrega  Cantidad  Costo plantación
0  Encino            4      1500             30000
1    Pino            4       950             19000
2  Encino            5      1400             28000
3    Pino            5      1000             20000
4  Encino            6       750             15000
5    Pino            6       850             17000
6  Encino            7       750             15000
7    Pino            7       550             11000

=== RUTAS ===
     Día                         Ruta  Duración_min  Unidades  \
0     4             18 → 6 → 17 → 18            95     

In [14]:
df_entregas

,Especie,Día entrega,Cantidad,Costo plantación
0,Encino,4,1500,30000
1,Pino,4,950,19000
2,Encino,5,1400,28000
3,Pino,5,1000,20000
4,Encino,6,750,15000
5,Pino,6,850,17000
6,Encino,7,750,15000
7,Pino,7,550,11000
